# Only loading Time

In [9]:
import torch
import time

time_counter = 0.0
A = torch.randn(10000,10000).half()
for i in range(10):
    
    # torch.cuda.synchronize()
    start_load = time.time()
    A = A.to("cuda",non_blocking=True)
    torch.cuda.synchronize()
    end_load = time.time()
    print("load time: ", end_load - start_load)
    A = A.to("cpu")
    time_counter += end_load - start_load
print("average load time: ", time_counter/10)


load time:  0.022943735122680664
load time:  0.023073196411132812
load time:  0.022989988327026367
load time:  0.023500680923461914
load time:  0.023050785064697266
load time:  0.02328801155090332
load time:  0.022878646850585938
load time:  0.023566007614135742
load time:  0.022873640060424805
load time:  0.02295851707458496
average load time:  0.02311232089996338


# Only computing time, related to data

In [11]:
time_counter = 0.0
B = torch.randn(10000,10000).half().to("cuda")
C = torch.randn(10000,10000).half().to("cuda")
for i in range(10):
    


    # torch.cuda.synchronize()
    start_compute = time.time()
    E = B.matmul(C)
    torch.cuda.synchronize()
    end_compute = time.time()
    time_counter += end_compute - start_compute
    print("compute time: ", end_compute - start_compute)
print("avg compute time: ", time_counter/10)


compute time:  0.03183341026306152
compute time:  0.03186988830566406
compute time:  0.031503915786743164
compute time:  0.03164196014404297
compute time:  0.03171515464782715
compute time:  0.031426191329956055
compute time:  0.0312654972076416
compute time:  0.031061649322509766
compute time:  0.030815839767456055
compute time:  0.03064441680908203
avg compute time:  0.03137779235839844


In [15]:
time_counter = 0.0
# A = torch.randn(10000,10000).half()
# B = torch.randn(10000,10000).half().to("cuda",non_blocking = False)
# D = torch.randn(10000,10000).half().to("cuda",non_blocking = False)
for i in range(10):
    
    start_combine = time.time()
    
    A = A.to("cuda",non_blocking=True)
    F = B.matmul(D)
    torch.cuda.synchronize()
    end_combine = time.time()
    time_counter += end_combine - start_combine
    print("combine time: ", end_combine - start_combine)
    A = A.to("cpu")
print("avg combine time: ", time_counter/10)


combine time:  0.06417250633239746
combine time:  0.06461691856384277
combine time:  0.0641329288482666
combine time:  0.06347894668579102
combine time:  0.0648643970489502
combine time:  0.06517481803894043
combine time:  0.06503534317016602
combine time:  0.0637204647064209
combine time:  0.06436300277709961
combine time:  0.06391119956970215
avg combine time:  0.06434705257415771


In [16]:
import time
import torch

time_counter = 0.0
B = torch.randn(10000, 10000).half().to("cuda", non_blocking=True)
D = torch.randn(10000, 10000).half().to("cuda", non_blocking=True)

for i in range(10):
    start = time.time()
    torch.cuda.synchronize()
    start_combine = time.time()

    A = A.to("cuda", non_blocking=True)
    F = B.matmul(D)
    
    torch.cuda.synchronize()
    end_combine = time.time()
    print("combine time: ", end_combine - start_combine)
    time_counter += end_combine - start_combine
    A = A.to("cpu")
print("avg combine time: ", time_counter / 10)

combine time:  0.06586003303527832
combine time:  0.06701278686523438
combine time:  0.06680750846862793
combine time:  0.06684398651123047
combine time:  0.0675055980682373
combine time:  0.06688666343688965
combine time:  0.06734323501586914
combine time:  0.06686615943908691
combine time:  0.06675100326538086
combine time:  0.06593847274780273
avg combine time:  0.06678154468536376


In [1]:
import time
import torch

time_counter = 0.0
A = torch.randn(10000, 10000).half()
B = torch.randn(10000, 10000).half().to("cuda", non_blocking=True)
D = torch.randn(10000, 10000).half().to("cuda", non_blocking=True)

# Create CUDA streams
stream1 = torch.cuda.Stream()
stream2 = torch.cuda.Stream()

for i in range(10):
    start_combine = time.time()

    with torch.cuda.stream(stream1):
        A = A.to("cuda", non_blocking=True)

    with torch.cuda.stream(stream2):
        F = B.matmul(D)

    # torch.cuda.synchronize()
    print(A[-1][-1])
    print()
    end_combine = time.time()
    print("combine time: ", end_combine - start_combine)
    time_counter += end_combine - start_combine
    A = A.to("cpu")
print("avg combine time: ", time_counter / 10)


combine time:  0.11871957778930664
combine time:  0.06522035598754883
combine time:  0.06349897384643555
combine time:  0.06327271461486816
combine time:  0.06302499771118164
combine time:  0.06320047378540039
combine time:  0.06348824501037598
combine time:  0.06352114677429199
combine time:  0.06276488304138184
combine time:  0.06293582916259766
avg combine time:  0.06896471977233887


# Conclusion
Pytorch could optimize recomputing and loading KV caches